In [33]:
import transformers
from transformers import pipeline
import datasets
import torch
import glob
import os
import pickle
from collections import defaultdict
import psutil
from loguru import logger
# import ray
import re
from pathlib import Path
from tqdm.notebook import tqdm

In [9]:
data = datasets.load_from_disk('/data/inet-large-scale-twitter-diffusion/ball4321/data_b/01_raw/data_cleaned_bt_split')

KeyboardInterrupt: 

In [ ]:
data

In [2]:
pipe = pipeline(
    task = 'ner',
    model='/data/inet-large-scale-twitter-diffusion/ball4321/data_b/04_models/roberta-large-NER',
    aggregation_strategy='simple',
    batch_size=250,
    device='cuda'
)

Some weights of the model checkpoint at /data/inet-large-scale-twitter-diffusion/ball4321/data_b/04_models/roberta-large-NER were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
data[0:30]['text']

['News broke this week of the surrender of Dominic Ongwen, a top commander in the Lord’s Resistance Army, to US forces in the Central African Republic.',
 'One of the five LRA commanders indicted by the ICC in 2005, Ongwen is also the most controversial of the Uganda indictments.',
 'While his surrender deals a serious blow to the LRA, whether the LRA’s victims will see justice remains unclear.',
 'Ongwen was a child when he was abducted by the LRA and grew up to become a notorious killer.',
 'But what are the legal and moral implications of trying a man for crimes against humanity when that man was conscripted as a child?',
 'The most notable aspect of the conflict is the LRA’s widespread use of abducted children as child soldiers; Ongwen himself is one of these child abductees.',
 'Abducted by the LRA in 1990 at the age of 10 while walking to school, over the years he rose through the ranks of LRA, ultimately becoming one of the senior commanders of the rebel group.',
 'There is litt

In [3]:
data2 = ['News broke this week of the surrender of Dominic Ongwen, a top commander in the Lord’s Resistance Army, to US forces in the Central African Republic.',
 'One of the five LRA commanders indicted by the ICC in 2005, Ongwen is also the most controversial of the Uganda indictments.']

In [4]:
# out = pipe(data[0:10]['text'])
out = pipe(data2)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
out[0]

[{'entity_group': 'PER',
  'score': 0.9999879,
  'word': 'Dominic Ongwen',
  'start': 41,
  'end': 55},
 {'entity_group': 'ORG',
  'score': 0.9999915,
  'word': 'Lord’s Resistance Army',
  'start': 80,
  'end': 102},
 {'entity_group': 'MISC',
  'score': 0.9921078,
  'word': 'US',
  'start': 107,
  'end': 109},
 {'entity_group': 'LOC',
  'score': 0.99999636,
  'word': 'Central African Republic',
  'start': 124,
  'end': 148}]

In [15]:
len(data)

42909472

In [25]:
out = []

batch_size=250

for batch_start in tqdm(range(0,1000000,batch_size)):
    batch_end = batch_start+batch_size
    # print(batch_start, batch_end)

    out.extend(pipe(data[batch_start:batch_end]['text']))

/tmp/ipykernel_2461379/3146181148.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_start in tqdm(range(0,1000000,batch_size)):


  0%|          | 0/4000 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 

In [27]:
out[-1]

[{'entity_group': 'PER',
  'score': 0.99998134,
  'word': 'Carina Coetzee',
  'start': 11,
  'end': 25},
 {'entity_group': 'PER',
  'score': 0.99999124,
  'word': 'Judy Sheehan',
  'start': 37,
  'end': 49}]

In [ ]:
# Start Ray cluster
ray.init(num_cpus=psutil.cpu_count(logical=True), ignore_reinit_error=True)

"""
The command ray.put(x) would be run by a worker process or by the driver process (the driver process is the one running your script). 
It takes a Python object and copies it to the local object store (here local means on the same node). 
Once the object has been stored in the object store, its value cannot be changed.
In addition, ray.put(x) returns an object ID, which is essentially an ID that can be used to refer to the newly created remote object. 
If we save the object ID in a variable with x_id = ray.put(x), 
then we can pass x_id into remote functions, 
and those remote functions will operate on the corresponding remote object.
"""
pipe_id = ray.put(pipe)

In [ ]:
# @ray.remote decorator enables to use this 
# function in distributed setting
@ray.remote
def predict(pipeline, text_data, label_names):
    return pipeline(text_data, label_names)

# Predict multipe texts on all available CPUs and time the inference duration
start = time.time()

# Run the function using multiple cores and gather the results
predictions = ray.get([predict.remote(pipe_id, text, ['motorcycle', 'baseball']) for text in test_data])

end = time.time()
print('Prediction time:', str(timedelta(seconds=end-start)))

# Stop running Ray cluster
ray.shutdown()

## NER Collate Sandbox

In [15]:
def extract_number(filename):
    # use only last part of filename
    filename=Path(filename).stem
    # Extract the number using regular expressions (regex)
    match = re.search(r'\d+', filename)
    # If a number is found, return it as an integer
    if match:
        return int(match.group())
    else:
        return 0

In [16]:
extract_number('/data/inet-large-scale-twitter-diffusion/ball4321/data_c/YT/raw_ner/raw_ner_0-999999.pkl')

0

In [14]:
Path('/data/inet-large-scale-twitter-diffusion/ball4321/data_c/YT/raw_ner/raw_ner_0-999999.pkl').stem

'raw_ner_0-999999'

In [34]:
filenames = glob.glob(os.path.join('/data/inet-large-scale-twitter-diffusion/ball4321/data_c/YT/raw_ner/', 'raw_ner_*.pkl'))

# function to sort glob data
def extract_number(filename):
    # use only last part of filename
    filename=Path(filename).stem
    # Extract the number using regular expressions (regex)
    match = re.search(r'\d+', filename)
    # If a number is found, return it as an integer
    if match:
        return int(match.group())
    else:
        return 0

# Sort the filenames based on the number
filenames_sorted = sorted(filenames, key=extract_number)

combined_result = defaultdict(set)
for counter, file in enumerate(filenames_sorted):
    # if counter % 100 == 0:
    logger.info(f'Processing {counter} of {len(filenames_sorted)}: {100*counter/len(filenames_sorted):.1f}%')
    with open(file,'rb') as f:
        data = pickle.load(f)

    for result, part_id in tqdm(data):
        processed_stories_id = part_id.split('_')[0]
        for item in result:
            if item['entity_group']=='PER':
                combined_result[processed_stories_id].add(item['word'])

2024-01-03 13:46:58.676 | INFO     | __main__:<module>:21 - Processing 0 of 12: 0.0%


  0%|          | 0/1000000 [00:00<?, ?it/s]

2024-01-03 13:47:05.402 | INFO     | __main__:<module>:21 - Processing 1 of 12: 8.3%


  0%|          | 0/4000000 [00:00<?, ?it/s]

2024-01-03 13:47:29.001 | INFO     | __main__:<module>:21 - Processing 2 of 12: 16.7%


...

  0%|          | 0/4000000 [00:00<?, ?it/s]

2024-01-03 13:51:02.560 | INFO     | __main__:<module>:21 - Processing 10 of 12: 83.3%


  0%|          | 0/4000000 [00:00<?, ?it/s]

2024-01-03 13:51:29.065 | INFO     | __main__:<module>:21 - Processing 11 of 12: 91.7%


  0%|          | 0/1909472 [00:00<?, ?it/s]

In [35]:
outpath = 'ner_id_to_ent.pkl'

with open(outpath, 'wb') as f:
    pickle.dump(combined_result, f)

logger.info(f'Saved to {outpath}')

2024-01-03 13:57:50.712 | INFO     | __main__:<module>:6 - Saved to ner_id_to_ent.pkl


In [36]:
inverted_dict = {}

# Iterating over the original dictionary
for key, values in combined_result.items():
    for value in values:
        # Append the key to the list of keys for this value
        if value in inverted_dict:
            inverted_dict[value].append(key)
        else:
            inverted_dict[value] = [key]

In [37]:
len(combined_result)

820798

In [40]:
outpath = 'ner_ent_to_id.pkl'

with open(outpath, 'wb') as f:
    pickle.dump(inverted_dict, f)

logger.info(f'Saved to {outpath}')

2024-01-03 14:06:13.564 | INFO     | __main__:<module>:8 - Saved to ner_ent_to_id.pkl


In [39]:
!pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.5 MB/s eta 0:00:0000:0100:01
